<h2>开始使用视频</h2>

<h3>目标</h3>

<ul>
    <li>学会读取视频、显示视频和保存视频</li>
    <li>学会从摄像头中捕捉并显示它</li>
    <li>你将要学习这两个函数：cv2.VideoCapture(), cv2.VideoWriter()</li>
</ul>

<h3>2.1 从摄像头中捕捉视频</h3>

<p>通常，我们必须用摄像头捕捉实时图像。OpenCv 为此提供了一个非常简单的接口。让我们使用摄像头捕获一段视频（我正使用我的笔记本电脑内置的摄像头），将它转换成灰度视频并显示它。这是是一个简单的开始。</P>

<p>为了捕获视频，你需要创建一个 VideoCapture 对象。它的参数可以是设备索引，也可以是视频文件的名称。设备索引就是指定的摄像头的编号。一般的笔记本中都有摄像头（比如在我的例子中）。所以我只传递 0 （或者 -1）。你可以通过传递 1 选择第二个摄像头等。之后，你就可以一帧一帧的捕获视频里。但是在最后不要忘了停止捕获视频。</P>

In [5]:
import numpy as np
import cv2

cap = cv2.VideoCapture(0)

while(True):
    #一帧一帧捕获视频 
    #ret的值为True 或 False 。frame 表示当前截取一帧的图片
    ret, frame = cap.read()
    
    #在这里，我们在每一帧上操作
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    #显示帧的结果
    cv2.imshow('frame',gray)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
#当一切结束时，释放资源
cap.release()
cv2.destroyAllWindows()

<p>cap.read() 返回一个布尔值（True ／ False）。如果帧读取是正确的，它会返回True。因此你可以通过检查它的返回值确定视频是否到达了文件尾。
    </p>

<p>有时，cap 不能正确的初始化捕获。这种情况下，上面的代码会报错。你可以检查它是否进行了初始化通过 cap.isOpened() 方法。如果返回值
是True，那么则没问题。否则，就使用 cap.open()。</p>

<p>你可以使用函数 cap.get(prold) 来获得视频的一些参数信息。这里propld 可以是1-18的任意整数。每一个数字表示视频的一个属性（如果它适用于该视频），可以在这里看到完整的详细信息：<a href="http://docs.opencv.org/modules/highgui/doc/reading_and_writing_images_and_video.html#videocapture-get">属性标识符</a>，其中的一些值可以用 cap.set(propld,value) 进行修改。value 是你想要的新值。</p>

<p>例如，我可以使用 cap.get(3) 和 cap.get(4) 来查看每一帧的宽和高。默认情况下得到的值是 640x480 。但是我们可以使用 ret=cap.set(3,320) 和 ret=cap.set(4,240) 来把宽和高改成320x240</p>

<h4>注意</h4>

当你的程序报错时，你首先应该检查的是你的摄像头能否在其他的程序中正常工作（比如Linux 下的 Cheese）

<h3>2.2 从文件中播放视频</h3>

与从摄像头中捕获相同，只要将摄像头的设备索引号改成视频的文件名称即可。在播放每一帧时，使用cv2.waitKey() 设置适当的持续时间。如果设置的太低视频就会播放的非常快，如果设置的太高，视频就会播放的特别慢（当然，这就是你如何使用慢镜头显示视频的方法）。25毫秒在正常情况下是可以的。

In [8]:
import numpy as np
import cv2
#改写了原文档中代码 改写后代码在视频播放完后，会自动退出循环
cap = cv2.VideoCapture('video1.avi')

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        cv2.imshow('frame',gray)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else :
        break
cap.release()
cv2.destroyAllWindows()

<h4>注意</h4>

你应该确保你已经正确安装了合适版本的 ffmpeg 或者 gstreamer。如果你装错了那就比较头疼了。

<h3>2.3 保存视频</h3>

我们捕获一个视频，然后一帧帧处理并且想保存这个视频。对于图像来说它是非常简单的，只需要使用 cv.imwrite().但是在这里，需要做更多的工作。

这次我们创建一个 VideoWriter 对象。我们应该指定一个输出文件名称（比如：out.avi) 然后，我们需要指定 FourCC 编码（下面将要介绍）。每秒的帧数和帧大小也需要确定。最后一个时 isColor 标志，如果时 True
，每一帧就是彩色图，否则就是灰色图。

FourCC 是一个四字节码，用来确定视频的编码格式。可用的编码列表可以从 fourcc.org 查到。这是依赖于平台，下面的编码器对我来说没问题。

<ul>
    <li>In Fedora: DIVX, XVID, MJPG, X264, WMV1, WMV2. (XVID 更加适合. MJPG 的结果是高尺寸的视频. X264 提供非常小的视频)</li>
    <li>In Windows: DIVX (有待测试和添加)</li>
    <li>In OSX :(我没有OSX的权限。有人能够填这张表吗？)</li>
    </ul>

FourCC 码是以下面的格式传给程序，以MJPG为例：cv2.VideoWriter_fourcc('M','J','P','G') 或者 cv2.VideoWriter_fourcc(*'MJPG)

下面的代码是从摄像投中捕获视频，沿水平方向旋转每一帧并保存它。

In [4]:
import numpy as np
import cv2

cap = cv2.VideoCapture(0)

# 定义编码格式并且创建 VideoWriter 对象
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi',fourcc, 20.0, (640,480))

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret==True:
        frame = cv2.flip(frame,0)

        # write the flipped frame
        out.write(frame)

        cv2.imshow('frame',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

# 如果工作结束则释放掉所有资源
cap.release()
out.release()
cv2.destroyAllWindows()

下面代码为：MAC 版本

In [21]:
# coding:utf-8
import cv2
import sys


cap = cv2.VideoCapture(0)
cap.set(3,640)
cap.set(4,480)
cap.set(1, 10.0)
#此处fourcc的在MAC上有效，如果视频保存为空，那么可以改一下这个参数试试, 也可以是-1
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
# 第三个参数则是镜头快慢的，10为正常，小于10为慢镜头
out = cv2.VideoWriter('output2.avi', fourcc,10,(640,480))
while True:
    ret,frame = cap.read()
    if ret == True:
        frame = cv2.flip(frame, 1)
        a = out.write(frame)
        cv2.imshow("frame", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
cap.release()
out.release()
cv2.destroyAllWindows()